In [1]:
import PyPDF2
import pandas
import os
import cv2
from PIL import Image
from paddleocr import PaddleOCR, draw_ocr

In [9]:
# Using padddleOCR
ocr = PaddleOCR(use_angle_cls = True, lang = 'en', page_num = 19)
pdf_path = 'CBP_Manual_1948_part1.pdf'
result = ocr.ocr(pdf_path, cls = True)

[2024/06/18 16:30:58] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=19, det_algorithm='DB', det_model_dir='/Users/steveshao/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/Users/steveshao/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=2

[[100.0, 76.0], [457.0, 79.0], [457.0, 107.0], [100.0, 104.0]]
('County business patterns.', 0.9999740123748779)
[[100.0, 112.0], [243.0, 112.0], [243.0, 134.0], [100.0, 134.0]]
('United States.', 0.9788848757743835)
[[100.0, 137.0], [956.0, 137.0], [956.0, 160.0], [100.0, 160.0]]
('[Washington] : Dept. of Commerce, Bureau of the Census for sale by the Supt. of Docs.,', 0.9932527542114258)
[[98.0, 162.0], [301.0, 162.0], [301.0, 183.0], [98.0, 183.0]]
('U.S. Govt. Print. Off.', 0.990537703037262)
[[100.0, 211.0], [640.0, 211.0], [640.0, 233.0], [100.0, 233.0]]
('http://hdl.handle.net/2027/mdp.39015018655723', 0.9970061182975769)
[[405.0, 358.0], [815.0, 362.0], [814.0, 447.0], [404.0, 444.0]]
('HathiTrust', 0.9997102618217468)
[[405.0, 825.0], [817.0, 832.0], [816.0, 883.0], [404.0, 876.0]]
('www.hathitrust.org', 0.9944560527801514)
[[409.0, 896.0], [805.0, 896.0], [805.0, 919.0], [409.0, 919.0]]
('Public Domain, Google-digitized', 0.9917900562286377)
[[404.0, 924.0], [817.0, 926.0], [

In [13]:
import fitz
from PIL import Image
import cv2
import numpy as np
imgs = []
PAGE_NUM = 19
with fitz.open(pdf_path) as pdf:
    for pg in range(0, PAGE_NUM):
        page = pdf[pg]
        mat = fitz.Matrix(2, 2)
        pm = page.get_pixmap(matrix=mat, alpha=False)
        # if width or height > 2000 pixels, don't enlarge the image
        if pm.width > 2000 or pm.height > 2000:
            pm = page.get_pixmap(matrix=fitz.Matrix(1, 1), alpha=False)
        img = Image.frombytes("RGB", [pm.width, pm.height], pm.samples)
        img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        imgs.append(img)
for idx in range(len(result)):
    res = result[idx]
    if res == None:
        continue
    image = imgs[idx]
    boxes = [line[0] for line in res]
    txts = [line[1][0] for line in res]
    scores = [line[1][1] for line in res]
    im_show = draw_ocr(image, boxes, txts, scores, font_path='simfang.ttf')
    im_show = Image.fromarray(im_show)
    im_show.save('result_page_{}.jpg'.format(idx))